In [6]:
%pip install pandas trino

Defaulting to user installation because normal site-packages is not writeable
Note: you may need to restart the kernel to use updated packages.


In [7]:
import pandas as pd
import trino
import warnings

warnings.simplefilter(action='ignore', category=UserWarning)

#### Connection

The connection seems to be a DBAPI2 object and behaves like other DB connection
objects (e.g. psycopg). We can use it directly from pandas.

In [8]:
conn = trino.dbapi.connect(
    host='localhost',
    port=8080,
    user='trino'
)

#### Query arbitrary data sources

We can directly hit multiple backend data sources with a single query. In this
example, we query a bunch of formats across multiple services.

* MinIO S3 object storage
  * Gzipped JSON
  * ORC
  * Parquet
* PostgreSQL server
  * Standard table

In [5]:
# Query multiple data sources directly.
df = pd.read_sql('''
  select p.name, p.age, o.color, c.make, c.model, v.value as est_purchase_price
  from minio.people.people p -- Gzipped JSON
  inner join minio.autos.owners o on p.name = o.person_name -- ORC
  inner join minio.autos.cars c on o.car_id = c.id -- Parquet
  left join postgresql.autos.car_values v -- PostgreSQL server
    on c.make = v.make and c.model = v.model and c.year = v.year
''', conn)
df

TrinoExternalError: TrinoExternalError(type=EXTERNAL, name=HIVE_METASTORE_ERROR, message="Failed connecting to Hive metastore: [hive-metastore.hive-system:9083]", query_id=20220319_190212_00000_5mywz)

#### Query a view

We can accomplish the same thing using a view, which we predefined in Trino.
This view is not materialized – but they can be – so it acts a virtual data set.

In [9]:
# Query a view.
df = pd.read_sql('''
  select * from minio.autos.car_values
  where make is not null
''', conn)
df

,name,age,color,make,model,value
0,Brendan,34,black,Mitsubishi,Outlander,23456.0
1,Blondie,55,red,Honda,Civic,14567.0
2,The Ginge,22,yellow,Ford,Taurus,NaN
